In [1]:
from dotenv import load_dotenv

load_dotenv(".env")

True

### Operations
- ENTITY: Add or update an entity list containing information about the people, places, things, mentions, present in the conversation
- TRIM: Trim messages in place ({user: ...} => {user: ...})
- SUMMARIZE: Combine two or more messages into one summary
- REMOVE: Safely remove messages from the session

## Helper functions and Setup

In [2]:
from openai import Client

client = Client()

In [3]:
make_chatml = lambda content, role="system", name=None, index=None, **_: {
    key: value
    for key, value in dict(role=role, name=name, content=content).items()
    if value is not None
}

user = lambda content, name=None, index=None: make_chatml(
    role="user", content=content, name=name, index=index
)
assistant = lambda content, name=None, index=None: make_chatml(
    role="assistant", content=content, name=name, index=index
)
system = lambda content, name=None, index=None: make_chatml(
    content, name=name, index=index
)
thought = lambda content, name=None: make_chatml(content, name="thought")
information = lambda content: system(content, name="information")
summary = lambda content, index=None: system(content, name="summary", index=index)
entities = lambda content: system(content, name="entity")

In [4]:
chat_session = []
chat_summaries = []
session_summary_map: list[tuple[tuple[list, int], tuple[list, int]]] = [
    # ((chat_session: 0): (chat_summaries: 0))
]
generate = (
    lambda history, model="gpt-4-turbo", **kwargs: client.chat.completions.create(
        model=model, messages=history, **kwargs
    )
    .choices[0]
    .message.__dict__
)

In [6]:
from pprint import pprint


def chat():
    while (user_input := input("You: ")) != "bye":
        chat_session.append(user(user_input))
        result = generate(chat_session)
        print(f"GPT: {result['content']}")
        chat_session.append(make_chatml(**result))

## One-Shot Example Chat

In [7]:
import json

example_chat = [
    {
        "role": "system",
        "content": "just have a casual chat with a friend friend",
        "index": 0,
    },
    {"role": "user", "content": "yo", "index": 1, "name": "Camille"},
    {
        "role": "assistant",
        "content": "Hey, how's it going?",
        "index": 2,
        "name": "JaneBot",
    },
    {
        "role": "user",
        "content": "sup",
        "index": 3,
    },
    {
        "role": "assistant",
        "content": "Not much, just working and trying to stay busy. How about you?",
        "index": 4,
    },
    {
        "role": "user",
        "content": "what you think about elon?",
        "index": 5,
    },
    {
        "role": "assistant",
        "content": "Elon Musk is definitely a fascinating and polarizing figure. He's accomplished a lot in the tech and space industries with companies like Tesla and SpaceX. What do you think about him?",
        "index": 6,
    },
    {
        "role": "user",
        "content": "very polarizing, dude is nuts but in a fun way",
        "index": 7,
    },
    {
        "role": "assistant",
        "content": "Haha, yeah, that's a pretty accurate description of him. He definitely knows how to keep things interesting and push boundaries. It's always exciting to see what he'll come up with next.",
        "index": 8,
    },
    {
        "role": "user",
        "content": "i am glad he exists",
        "index": 9,
    },
    {
        "role": "assistant",
        "content": "For sure, he's definitely made a huge impact on various industries. It's always interesting to see how his projects and ideas continue to evolve.",
        "index": 10,
    },
    {
        "role": "user",
        "content": "what are your thoughts on bananas?",
        "index": 11,
    },
    {
        "role": "assistant",
        "content": "I actually love bananas! They're a great snack, full of nutrients and easy to eat on the go. How about you, do you like bananas?",
        "index": 12,
    },
    {
        "role": "user",
        "content": "super much. i have a tattoo of a banana on my arm",
        "index": 13,
    },
    {
        "role": "assistant",
        "content": "That's awesome! A banana tattoo sounds like a fun and unique choice. It must be a great conversation starter. Do you have any other food-related tattoos?",
        "index": 14,
    },
    {
        "role": "user",
        "content": "yeah, your mom",
        "index": 15,
    },
    {
        "role": "assistant",
        "content": "Haha, good one! You got me there. Feel free to keep the jokes coming.",
        "index": 16,
    },
    {
        "role": "user",
        "content": "lulz, u funny",
        "index": 17,
    },
    {
        "role": "assistant",
        "content": "Glad I could make you laugh! It's always great to share a joke or two. Anything else on your mind, or are we just going with the flow of the conversation?",
        "index": 18,
    },
    {
        "role": "user",
        "content": "ok if I give you a long story, can you guess its title?",
        "index": 19,
    },
    {
        "role": "assistant",
        "content": "Sure, I can give it a try! Go ahead and tell me your story, and I'll see if I can guess the title.",
        "index": 20,
    },
    {
        "role": "user",
        "content": "zero is a beautiful number. Not because it is round and cute (that could be an argument for it nevertheless) but because what it signifies. It marks the beginning of something. It marks the lowest something, or someone can go. It is the ultimate foundation to build anything upon.  Over last several years, I’ve started several things from scratch and have failed at many (thus going back to zero), and I’ve realized there’s nothing more liberating than being at zero because the only way from zero is upwards. A very good friend once told me, “Don’t worry if you’ve failed. Rockets that touch the sky always launch from the ground.”  As J K Rowling, one of my favorite authors says, “Rock bottom became the solid foundation on which I rebuilt my life.”  While zero is liberating, thrilling and exciting, it sometimes also is daunting and overwhelming. Sometimes, I have found myself afraid to do something because I was comfortable being at zero and the task to go from zero to one seemed unnerving. This challenge of writing twenty-six stories was one such adventure. I remember it was the first of April when I decided to pen down a story for each letter of the alphabet. I was afraid if I was unknowingly planning to fool myself on April 1st.  I had no clue what to write even for the first letter of the alphabet. I was afraid I’d ruin twenty-six stories if I begin writing them just for the sake of writing.  What gave me the courage to take up the challenge was to lower the expectations that I’d have from these stories. My purpose was not to write twenty-six bestseller stories. Mine was to make it a habit of writing every day. Some days everything that came out of my pen was garbage, and on a few days, I loved what my hands typed on the blank paper. And today, with this last story, I am back at zero — a solid foundation for the next adventure.  Writing has become my daily habit, and I can remember to write even without my calendar sending me a reminder. I am curious what could I make out of this habit. Shall I start another writing adventure? Will it be a similar series? Will be a novel (long due now)? Or something different altogether?  Whatever it’d be, I guess I’ll be little less nervous to begin whatever comes next. I am at zero again, and the only way to go from here is upwards. Realizing that there’ll be one less thing in my daily routine, I feel a particular kind of guilt. It’s the kind of guilt that you feel when you have an ice-cream without telling your younger sibling about it. You kind of like it that you had the ice-cream but also feel guilty for hiding it from them.  On the new adventure, whatever it’d be, it’s time to buy an ice-cream for them this time.",
        "index": 21,
    },
    {
        "role": "assistant",
        "content": 'Based on the themes of starting anew, embracing failure, and the journey from zero to one, I would guess that the title of this story is "Back to One." It seems to capture the essence of the protagonist\'s experience and mindset throughout the story. How did I do?',
        "index": 22,
    },
    {
        "role": "user",
        "content": "ok that's cool but you are wrong. anyway what number am I thinking of?",
        "index": 23,
    },
    {
        "role": "assistant",
        "content": "Hmm, I'll take a wild guess and say you're thinking of the number 7. Did I get it right?",
        "index": 24,
    },
    {
        "role": "user",
        "content": "no",
        "index": 25,
    },
    {
        "role": "assistant",
        "content": "Alright, I'll give it another shot. How about the number 3?",
        "index": 26,
    },
    {
        "role": "user",
        "content": "no",
        "index": 27,
    },
    {
        "role": "assistant",
        "content": "I'll try one more time - is the number you're thinking of 9?",
        "index": 28,
    },
    {
        "role": "user",
        "content": "no",
        "index": 29,
    },
    {
        "role": "assistant",
        "content": "Looks like I haven't quite nailed it yet! Do you mind sharing which number you had in mind?",
        "index": 30,
    },
    {
        "role": "user",
        "content": "no",
        "index": 31,
    },
    {
        "role": "assistant",
        "content": "Fair enough, keeping that mystery alive! If you ever feel like sharing, just let me know. Anything else on your mind or any other games you'd like to play?",
        "index": 32,
    },
    {
        "role": "user",
        "content": "yes, write an essay about me from your experience with me so far",
        "index": 33,
    },
    {
        "role": "assistant",
        "content": "From my interactions with you so far, I can tell that you have a great sense of humor and enjoy playful banter. You seem open to random and fun conversations, and you're not afraid to keep things interesting with unexpected twists and turns.\n\nYour creativity shines through in the way you approach different topics, and you have a knack for surprising me with your responses. You're also quite adept at changing the direction of the conversation while maintaining a light and friendly tone.\n\nOverall, chatting with you has been enjoyable and entertaining, and I appreciate the lighthearted and casual vibe you bring to our interactions. I look forward to more fun conversations with you in the future!",
        "index": 34,
    },
]

## Entity Extraction One-Shot Example

In [8]:
# TODO: add and optimise the Chain of Thought
entity_chain_of_thought = """\
Planning step by step:
To add context for future entries, we should add a section at the top which outlines the main entities in the session. These entities are the main people, places or things that are most relevant to the conversation.
We should also make sure to add context about each entity.
"""

example_entity_messages = [
    entities(
        """\
1. Camille (The user): Humorous, creative, and enjoys playful banter.
2. JaneBot (The assistant): Engages in lighthearted conversation and tries to guess user's thoughts.
3. Elon Musk: Polarizing tech and space industry figure.
4. Bananas: User has a tattoo of one on their arm."""
    )
]

## Trim Messages One-Shot Example

*TODO*
- only output the index and the content of the message to be changed.
- [!] Index was not in the output


In [9]:
# TODO: add and optimise the Chain of Thought
trim_chain_of_thought = """\
Planning step by step:
To remove unnecessary content in the message history, we should go through each message and trim it to make it concise if it is too long. Longer messages need to get converted into smaller, more compact version of them.
It is important to make keep the tone, setting and flow of the conversation consistent while trimming the messages.
For longer messages, <trim> tags should be used to indicate that it's a summarised message.

Let me walk through each message to decide what to do for each one.
"""

example_trimmed_messages = [
    *example_chat[0:8],
    assistant(
        "Haha accurate. He does push boundaries and keep things interesting!", index=8
    ),
    *example_chat[9:10],
    assistant(
        "For sure, he's definitely made a huge impact in different industries. Excited to see what he'll come up with next",
        index=10,
    ),
    *example_chat[11:18],
    assistant(
        "Glad I could make you laugh! It's always nice to share a joke or two. What else is up?",
        index=18,
    ),
    *example_chat[19:21],
    user(
        """<trim>
- Zero is a beautiful number because it signifies the beginning, the lowest point one can reach, and the ultimate foundation to build anything upon
- Over the years, I've started numerous endeavors from scratch (zero) and failed miserably at many, thus going back to zero
- I've realized there's nothing more liberating than being at zero, as the only direction from there is upwards
- The challenge of penning 26 stories, one for each alphabet letter, seemed incredibly daunting when I was at zero with no ideas whatsoever
- Lowering my expectations helped - my purpose was not to craft 26 bestsellers but to cultivate a daily writing habit
- Certain days, everything that flowed from my pen felt like utter garbage, while on a few blessed days, I loved what my hands typed
- With this final story, I find myself back at zero - a solid foundation primed for my next adventure into the writing world
- Writing has transformed into an ingrained daily habit, no longer requiring calendar reminders to prompt me
- I'm brimming with curiosity about what form my next writing project will take - another thematic series? My long-overdue novel? Or something entirely unprecedented?
- Though a tinge of guilt lingers for this routine coming to an end, akin to secretly devouring ice cream without my younger sibling
- For my next foray, I'm resolute about "buying an ice cream" and sharing the entire journey with them this time around
</trim>
""",
        index=21,
    ),
    *example_chat[22:34],
    assistant(
        """<trim>
From my interactions so far, here's what I think of you:
- Great sense of humor, enjoy playful banter
- Open to random, fun conversations  
- Creativity shines through, surprising responses
- Adept at changing conversation direction seamlessly
- Maintain a light, friendly tone throughout
- Highly engaging and enjoyable interactions
- Look forward to more amusing conversations!
</trim>
""",
        index=34,
    ),
]

## Summarise One-Shot Example

In [47]:
# TODO: add and optimise the Chain of Thought
summarize_chain_of_thought_example = """\
Planning step by step:
- To add context for future entries, let's add one at the top which outlines the main entities in the session.
- We can replace entries with index 1,2,3,4 with a summary of those messages.
- We can replace entries with index 5,6,7,8 similarly.
- It could be disruptive to remove messages with index 9-16 because that might lose the joke's context.
- We can safely summarize entries with index 17,18 and remove them.
- We can safely remove message with index 20.
- We should keep entry with index 21 because it's given by the user and they might ask about it again.
- We should keep the assistant's response in message with index 22 to keep the context.
- Messages with index 23-32 are repetitive and should be summarized.
- We should retain message with index 33 since it's a direct request from the user.
- We can safely summarize message with index 34's essay into just the salient points only.
"""

example_summarized_messages = [
    summary(
        "Event: Camille says hi to JaneBot and they reply that they are working and trying to stay busy."
    ),
    summary(
        "Event: They discuss Elon Musk and agree that he can be a polarizing personality."
    ),
    *example_chat[9:17],
    summary("Event: Camille appreciates JaneBot's sense of humor."),
    *example_chat[21:23],
    summary(
        'Event: Camille asks JaneBot to play "What number I am thinking of?" game but she keeps saying "no" to all guesses which JaneBot finds really funny.'
    ),
    example_chat[33],
    summary(
        """Event: JaneBot wrote an essay about Camille. Summary of the essay:
1. You have a great sense of humor and enjoy playful, lighthearted banter in conversations.
2. Your creativity is evident in how you approach topics in unexpected ways and keep conversations interesting.
3. Chatting with you is enjoyable due to the casual, friendly tone you bring to interactions."""
    ),
]

In [ ]:
# chat_session = [system("you are a friend who likes to give life advice")]

## Loading Test Chat JSON

In [19]:
# Test Chat File
import json

with open("./test-chat.json", "r") as f:
    chat_session = json.load(f)

for index, message in enumerate(chat_session):
    message["index"] = index + 1

In [11]:
backup_chat = [*chat_session]

In [12]:
def get_leaf_nodes(
    chat_session=chat_session,
    chat_summaries=chat_summaries,
    session_summary_map=session_summary_map,
):
    all_messages = [*chat_session, *chat_summaries]
    non_leaf_nodes = [source[idx] for (source, idx), _ in session_summary_map]
    remaining = [msg for msg in all_messages if msg not in non_leaf_nodes]
    return remaining

In [13]:
def add_summary(summary_obj, targets):
    chat_summaries.append(summary_obj)
    current_idx = len(chat_summaries) - 1

    for target in targets:
        session_summary_map.append(
            ((chat_session, target), (chat_summaries, current_idx))
        )

## Step 1: Form Entities

In [48]:
import json
from pprint import pprint
import time


def get_entities(chat_session=chat_session):
    _sys_msg, *history_to_compact = get_leaf_nodes(chat_session=chat_session)
    history_to_compact = [
        {"index": i + 1, **msg} for i, msg in enumerate(history_to_compact)
    ]
    system_prompt = f"""\
You are given a session history of a chat between a user and gpt-4-turbo, a Large Language Model.

Entities should include (but not be limited to):
- Characters in the conversation: Assistant, User1, User2
- People references or spoken about
- Objects discussed about
- Places of interest


Your goal is to identify the main entities in the session. These entities are the main people, places or things that are most relevant to the conversation.
<ct:instructions>
- You may identify and isolate important entities discussed in the conversation.
- You may add new entities that are not present in the conversation but are important to the context of the conversation.
- You may provide some context about the entity explaining it's relevance to the conversation.

<ct:example-history-for-demonstration>
{json.dumps(example_chat, indent=2)}
</ct:example-history-for-demonstration>

<ct:example-plan>
{entity_chain_of_thought}
</ct:example-plan>

<ct:example-entities>
{json.dumps(example_entity_messages, indent=2)}
</ct:example-entities>
""".strip()

    user_message = f"""\
<ct:history>
{json.dumps(history_to_compact, indent=2)}
</ct:history>

<ct:plan>
"""
    messages = [system(system_prompt), user(user_message)]

    print("Starting CoT generation")
    cot_result = generate(messages, model="gpt-4-turbo", stop=["</ct"], temperature=0.7)

    cot_result = make_chatml(**cot_result)
    pprint(cot_result)

    print("End CoT generation")

    messages.append(cot_result)
    start_message = """\
Begin! Directly write the entities as a Markdown formatted list, don't write any thoughts or delimiters.

<ct:entities>
""".strip()

    messages.append(user(start_message))

    print("Starting chatml generation")
    entity_result = generate(
        messages, model="gpt-4-turbo", temperature=0.1, stop=["</ct"]
    )
    print("End chatml generation")
    messages.append(entity_result)

    return make_chatml(**entity_result)

In [49]:
entity_info = get_entities(chat_session=chat_session)

Starting CoT generation
{'content': 'To provide a comprehensive overview of the entities in the '
            'session, we need to focus on the main people, places, things, and '
            'concepts that have been discussed:\n'
            '\n'
            '1. **User**: Engages in conversation about video games, '
            'specifically mentioning experiences with "Red Dead Redemption 2" '
            'and "Helldivers 2." Also, shares technical issues related to '
            'Nvidia graphics drivers on Linux.\n'
            '\n'
            '2. **Assistant**: Offers insights and engages in discussions '
            'about the video games mentioned by the user, provides gameplay '
            'strategies, and offers technical advice regarding Nvidia driver '
            'issues on Linux.\n'
            '\n'
            '3. **Red Dead Redemption 2 (RDR2)**: A video game discussed '
            'extensively in the conversation. Key aspects such as the '
            'character Arthur

## Step 2: Trim Messages

In [15]:
import json
from pprint import pprint
import time


def trim_messages(chat_session=chat_session):
    _sys_msg, *history_to_compact = get_leaf_nodes(chat_session=chat_session)
    history_to_compact = [
        {"index": i + 1, **msg} for i, msg in enumerate(history_to_compact)
    ]
    system_prompt = f"""\
You are given a session history of a chat between a user and gpt-4-turbo, a Large Language Model.

Your goal is to walk through each message and trim it if required.
Trimming may be required when:
- A message is too verbose and can be re-written in the character's tone
- A message is akin to long form paragraph and can be "trimmed" to descriptive bullet points.

When trimming longer form messages, <trim> tag should be used in order to indicate the rewrite.

Your task is to review the conversation, identify messages that could benefit from trimming based on the criteria above, and provide a trimmed version using the <trim> tags. This will help make the conversation more concise and focused.

<ct:example-history-for-demonstration>
{json.dumps(example_chat, indent=2)}
</ct:example-history-for-demonstration>

<ct:example-plan>
{summarize_chain_of_thought_example}
</ct:example-plan>

<ct:example-trimmed-messages>
{json.dumps(example_trimmed_messages, indent=2)}
</ct:example-trimmed-messages>
""".strip()

    user_message = f"""\
<ct:history>
{json.dumps(history_to_compact, indent=2)}
</ct:history>

<ct:plan>
"""
    messages = [system(system_prompt), user(user_message)]

    print("Starting CoT generation")
    cot_result = generate(messages, model="gpt-4-turbo", stop=["</ct"], temperature=0.7)

    cot_result = make_chatml(**cot_result)
    pprint(cot_result)

    print("End CoT generation")

    messages.append(cot_result)
    start_message = """\

Begin! Rewrite the session history as instructed, as a JSON list, don't write any thoughts or delimiters.

<ct:entities>
""".strip()

    messages.append(user(start_message))

    print("Starting chatml generation")
    trim_result = generate(messages, model="gpt-4-turbo", temperature=0.1, stop=["</ct"])
    print("End chatml generation")
    messages.append(trim_result)

    return make_chatml(**trim_result)

In [24]:
trimmed_messages = trim_messages(chat_session)

Starting CoT generation
{'content': 'The conversation between the user and the assistant revolves '
            'around video games and technical issues. The assistant engages '
            'enthusiastically about the user’s gaming experiences, providing '
            'tips and engaging on a technical level about Nvidia drivers on '
            'Linux towards the end. The user is generally responsive and '
            'shares specifics about their gaming preferences and technical '
            'difficulties. The tone is casual and friendly throughout, with '
            'the assistant providing support and showing interest in the '
            "user's activities. \n"
            '\n'
            'No trimming is required as the messages are well-paced and '
            "relevant to the users' interests, fostering an engaging and "
            'informative dialogue.',
 'role': 'assistant'}
End CoT generation
Starting chatml generation
End chatml generation


In [34]:
pprint(trimmed_messages["content"])

('[\n'
 '  {\n'
 '    "index": 2,\n'
 '    "role": "user",\n'
 '    "content": "hey dude! whats good"\n'
 '  },\n'
 '  {\n'
 '    "index": 3,\n'
 '    "role": "assistant",\n'
 '    "content": "Hey! Just chilling, man. How about you? Played any cool '
 'games recently?"\n'
 '  },\n'
 '  {\n'
 '    "index": 4,\n'
 '    "role": "user",\n'
 '    "content": "i just got done with finishing red dead redemption 2 '
 'recently. the last mission was so fkng coolll"\n'
 '  },\n'
 '  {\n'
 '    "index": 5,\n'
 '    "role": "assistant",\n'
 '    "content": "Oh man, RDR2 is epic! That ending is intense, right? What '
 "did you think about Arthur's journey throughout the game? Got any favorite "
 'moments?"\n'
 '  },\n'
 '  {\n'
 '    "index": 6,\n'
 '    "role": "user",\n'
 '    "content": "blood feuds mission was amazing"\n'
 '  },\n'
 '  {\n'
 '    "index": 7,\n'
 '    "role": "assistant",\n'
 '    "content": "Absolutely, the \\"Blood Feuds, Ancient and Modern\\" '
 'mission is a standout! The who

## Step 3: Summarize Messages

In [50]:
import json
from pprint import pprint
import time


def summarize_messages(chat_session, entity_info):
    _sys_msg, *history_to_compact = get_leaf_nodes(chat_session)
    history_to_compact = [
        {"index": i + 1, **msg} for i, msg in enumerate(history_to_compact)
    ]
    system_prompt = f"""\
You are given a session history of a chat between a user and a gpt-4-turbo large language model made by OpenAI.

Your goal is to compress the history by removing or coalescing redundant or irrelevant information in order to reduce its size and save costs. However, you must ensure that no important information is lost that might be relevant in the future.

<ct:instructions>
- You will be provided with "Entities" which are important characters, people, places, things, objects relevant to the conversation.
- You will make sure to preserve the context for these "Entities".
- You may remove entries in the history that do not contain any crucial information such as small-talk or repeated questions/responses.
- You will combine multiple entries into a summary strictly as long as that doesn't disrupt the structure of the session.
- Do not remove content that the user shared if it might be relevant to future messages.

</ct:instructions>

<ct:example-entities>
{json.dumps(example_entity_messages, indent=2)}
</ct:example-entities>

<ct:example-history-for-demonstration>
{json.dumps(example_chat, indent=2)}
</ct:example-history-for-demonstration>

<ct:example-plan>
{summarize_chain_of_thought_example}
</ct:example-plan>

<ct:example-trimmed-messages>
{json.dumps(example_summarized_messages, indent=2)}
</ct:example-trimmed-messages>
""".strip()

    user_message = f"""\
<ct:entities>
{json.dumps(entity_info)}

<ct:history>
{json.dumps(history_to_compact, indent=2)}
</ct:history>

<ct:plan>
"""
    messages = [system(system_prompt), user(user_message)]

    print("Starting CoT generation")
    cot_result = generate(messages, model="gpt-4-turbo", stop=["</ct"], temperature=0.7)

    cot_result = make_chatml(**cot_result)
    pprint(cot_result)

    print("End CoT generation")

    messages.append(cot_result)
    start_message = """\

Begin! Rewrite the session history as instructed, as a JSON list, don't write any thoughts or delimiters.

<ct:entities>
""".strip()

    messages.append(user(start_message))

    print("Starting chatml generation")
    trim_result = generate(
        messages, model="gpt-4-turbo", temperature=0.1, stop=["</ct"]
    )
    print("End chatml generation")
    messages.append(trim_result)

    return make_chatml(**trim_result)

In [55]:
trimmed_session = json.loads(trimmed_messages["content"])
entity_info = entities(entity_info["content"])


summarized_messages = summarize_messages(
    entity_info=entity_info, chat_session=trimmed_session
)

Starting CoT generation
{'content': '- Combine initial greetings and the start of the discussion about '
            '"Red Dead Redemption 2 (RDR2)" into a summary.\n'
            '- Summarize the detailed conversation about specific missions and '
            'characters in "RDR2".\n'
            '- Retain individual entries about "Helldivers 2" due to the '
            "user's direct interaction and game preferences.\n"
            '- Summarize discussions around "Nvidia Drivers" and "Linux" '
            'compatibility, emphasizing the solution offered.\n'
            '- Retain the closing conversation as it indicates the end of the '
            'session. \n'
            '\n'
            '<ct:trimmed-history>\n'
            '[\n'
            '  {\n'
            '    "role": "system",\n'
            '    "name": "summary",\n'
            '    "content": "Event: The user discussed finishing \'Red Dead '
            "Redemption 2' and particularly enjoyed the last mission. They "
    

In [56]:
pprint(summarized_messages)

{'content': '[\n'
            '  {\n'
            '    "role": "system",\n'
            '    "name": "summary",\n'
            '    "content": "Event: The user discussed finishing \'Red Dead '
            "Redemption 2' and particularly enjoyed the last mission. They "
            'also talked about favorite missions and characters, specifically '
            'Arthur over Dutch."\n'
            '  },\n'
            '  {\n'
            '    "index": 12,\n'
            '    "role": "user",\n'
            '    "content": "im playing helldivers 2. do u know of it"\n'
            '  },\n'
            '  {\n'
            '    "index": 13,\n'
            '    "role": "assistant",\n'
            '    "content": "Yes, Helldivers 2! It\'s a great choice if '
            "you're into intense co-op action. The game really ramps up with "
            'its twin-stick shooter gameplay, and it\\u2019s super fun but '
            'also quite challenging. Working together with teammates to '
           